# Step #1 Imports and Authentication

In [9]:
# A tutorial for this file is available at www.relataly.com
# Tested with Python 3.9.13, 1.2.0, Pandas 1.3.4, OpenAI 0.27.3, Tweepy 4.13.0, Requests 2.26.0, 

import logging
import pandas as pd
import openai
import tweepy
import csv
import requests

# Set API Keys and Authentification
logging.info('Setting NewsAPI API Key')
NEWSAPI_API_KEY = <your API key> # replace with own API key

logging.info('Setting OpenAI API Key')
openai.api_key = <your API key> # replace with own API key

logging.info('Setting Twitter API Key')
auth=tweepy.OAuthHandler(<your API key>,
                         <your API secret>)
auth.set_access_token(<your access token>,
                      <your access secret>)
twitter_api=tweepy.API(auth)

CSV_NAME = 'news_log.csv'

# Step #2 Function for News API

In [10]:
#### NewsAPI
def fetch_news(number=10):
    # Fetch tech news from NewsAPI
    url = f"https://newsapi.org/v2/top-headlines?country=us&category=technology&apiKey={NEWSAPI_API_KEY}"
    response = requests.get(url).json()
    news_items = response["articles"]
    df = pd.DataFrame(news_items)
    df = df[["title", "description", "url"]].dropna()
    return df.head(number)

# Step #3 OpenAI Functions for News Relevance and Tweet Generation

In [11]:
#### OpenAI Engine
def openai_request(instructions, task, sample = [], model_engine='gpt-3.5-turbo'):
    prompt = [{"role": "system", "content": instructions }, 
              {"role": "user", "content": task }]
    prompt = sample + prompt
    completion = openai.ChatCompletion.create(model=model_engine, messages=prompt, temperature=0.5, max_tokens=300)
    return completion.choices[0].message.content


#### Define OpenAI Prompt for news Relevance
def select_relevant_news_prompt(news_articles, topic):    
    instructions = f'Your task is to examine a list of News Titles and return a list of boolean values that indicate which of the News Titles are relevant to a provided list of topics. \
    Return a list of True or False values that indicate the relevance for the News Titles.'
    task =  f"News Titles: {news_articles}. /n Topics: {topic}." 
    sample = [{"role": "user", "content": f"News: ['new AI model available from Nvidia', \
               'We Exploded the AMD Ryzen 7 7800X3D & Melted the Motherboard', \
               'The Lara Croft Collection For Switch Has Been Rated By The ESRB - Nintendo Life']. /n \
               Topics: {topic}. "},
                {"role": "assistant", "content": "[True, False, False]"}]
    return instructions, task, sample


def check_previous_posts_prompt(title, old_posts):    
    instructions = f'Your objective is to compare a news title with a list of previous news posts and determine whether it covers a topic that was previously covered by another article. \
        The goal is to prevent posting about the same news twice. If a match is found, just return "True" indicate that the news was already covered; otherwise, return "False"'
    task =  f'{title}. Previous News Posts: {old_posts}.'
    sample = [{"role": "user", "content": "Nvidia launches new AI model. Previous News Posts: ['new AI model available from Nvidia', 'We Exploded the AMD Ryzen 7 7800X3D & Melted the Motherboard', 'The Lara Croft Collection For Switch Has Been Rated By The ESRB - Nintendo Life']."},
    {"role": "assistant", "content": "True"},
    {"role": "user", "content": "Big Explosion of an AMD Ryzen 7 - wow this explodes. Previous News Posts: ['new AI model available from Nvidia', 'The Lara Croft Collection For Switch Has Been Rated By The ESRB - Nintendo Life']."},
    {"role": "assistant", "content": "False"},
    {"role": "user", "content": "Google Plans More Visual, Snackable Search, With Videos, AI, Report Says - CNET. Previous News Posts: ['new AI model available from Nvidia', 'The Lara Croft Collection For Switch Has Been Rated By The ESRB - Nintendo Life']."},
    {"role": "assistant", "content": "False"}]
    return instructions, task, sample


#### Define OpenAI Prompt for News Tweet
def create_tweet_prompt(title, description, tiny_url):
    instructions = f'You are a twitter user that creates tweets with a maximum length of 280 characters.'
    task = f"Create an informative tweet on twitter based on the following news title and description. \
        The tweet must use a maximum of 280 characters. \
        Include the {tiny_url}. But do not include any other urls.\
        Title: {title}. \
        Description: {description}. \
        Use hashtags to reach a wider audience. \
        Do not include any emojis in the tweet"
    return instructions, task

# Step #4 Functions for Tweeting

In [12]:
def check_tweet_length(tweet):
    if len(tweet) > 280:
        print(f'Tweet too long: {len(tweet)}')
        return False
    else:
        print(f'Tweet length OK: {len(tweet)}')
        return True


def create_tweet(title, description, url):
    # create tiny url
    tiny_url = create_tiny_url(url)

    # define prompt for tweet creation
    instructions, task = create_tweet_prompt(title, description, tiny_url)
    tweet = openai_request(instructions, task)
    tweet = tweet.replace('"', '')

    # check tweet length and post tweet
    if check_tweet_length(tweet):
            print(f'Creating tweet: {tweet}')
            status = twitter_api.update_status(tweet)
            logging.info(f'Tweet posted: {status.id}')
    else: 
        status = 'error tweet too long'
    return status


def create_tiny_url(url):
    response = requests.get(f'http://tinyurl.com/api-create.php?url={url}')
    return response.text

# Step #5 Bringing it All Together - Running the Bot

In [13]:
def main_bot():
    # Read the old CSV data
    # try opening the csv file and creeate it if it does not exist
    try:
        df_old = pd.read_csv(CSV_NAME)
    except:
        # create the csv file
        df_old = pd.DataFrame(columns=['title'])
        df_old.to_csv(CSV_NAME, index=False)

    # Fetch news data
    df = fetch_news(10)

    # Check the Relevance of the News and Filter those not relevant
    relevant_topics ='[AI, Python Programming, Machine Learning, Applications of Artificial Intelligence, Software Engineering]'
    instructions, task, sample = select_relevant_news_prompt(list(df['title']), relevant_topics)
    relevance = openai_request(instructions, task, sample)
    relevance = eval(relevance)

    df = df[relevance]

    if len(df) > 0:
        for i, row in df.iterrows():
            title = row['title']
            description = row['description']
            url = row['url']
            doublicate_check = previous_post_check(title, list(df_old.tail(10)['title']))
            if doublicate_check == False:            
                # Check if the title has already been tweeted
                if title not in df_old.title.values:
                    # Create a tweet
                    response = create_tweet(title, description, url)
                    print(response)

                    print(f"Tweeted: {title}")
                    with open(f'{CSV_NAME}', 'a', newline='') as file:
                        writer = csv.writer(file)
                        writer.writerow([title])
                    
                else: 
                    print(f"Already tweeted: {title}")
                    logging.info(f"Already tweeted: {title}")
            else: 
                print(f"Doublicatecheck True: {title}")
                logging.info(f"Context Doublicate: {title}")
                with open(f'{CSV_NAME}', 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([title])
                    
    else: 
        print("No news articles found")
        logging.info("No news articles found")

main_bot()

,title,description,url,relevance
0,The iPhone 14 Pro is a demonstration of what o...,Apple’s iPhone 14 Pro is getting more useful w...,https://www.theverge.com/2023/5/5/23711698/iph...,False
1,Huge update is launched for billions of Gmail ...,Google has unveiled a huge update that signals...,https://www.dailymail.co.uk/sciencetech/articl...,False
2,JBL Tour Pro 2 review: Welcome to the smart ca...,JBL threw a ton of features in the Tour Pro 2 ...,https://www.engadget.com/jbl-tour-pro-2-review...,False
3,Tech Issues Are Plaguing Next-Gen Games - Next...,In this week's monster-sized episode of Next-G...,https://www.youtube.com/watch?v=WqF9V1dwBW0,False
4,Xbox boss raises eyebrows with claim Microsoft...,As the fallout from Redfall's release continue...,https://www.eurogamer.net/xbox-boss-raises-eye...,False
5,"May 2023 update lands on Galaxy S22, starting ...",Samsung is one week into the May 2023 security...,https://www.sammobile.com/news/may-2023-update...,False
6,What to expect at Google I/O 2023 - Engadget,Google is set to introduce Android 14 and new ...,https://www.engadget.com/what-to-expect-at-goo...,False
7,PSA: you've got just one weekend left to claim...,You’ve got until May 9th to claim the dozen-pl...,https://www.theverge.com/2023/5/5/23712262/son...,False
8,How to Lock Specific iPhone Apps Behind Face I...,"In the interests of enhanced privacy, some thi...",https://www.macrumors.com/how-to/automatically...,False
9,Microsoft is allegedly working with AMD to exp...,Microsoft is allegedly providing engineering r...,https://www.theverge.com/2023/5/5/23712242/mic...,True


Tweet too long: 335
error tweet too long
Tweeted: Microsoft is allegedly working with AMD to expand into AI chips - The Verge


In [14]:

CSV_NAME = 'news_log.csv'
df_old = pd.read_csv(CSV_NAME)
import pandas as pd
title = 'katze'
df = pd.DataFrame({'title': [title]})
df
df_old = pd.concat([df_old, df], ignore_index=True)
df_old

,title
0,Researchers See Through a Mouse's Eyes by Deco...
1,Google DeepMind CEO says human-level AI will b...
2,AMD jumps 9% on report Microsoft is collaborat...
3,Next week's Google I/O event may change Samsun...
4,"Microsoft barrels ahead with AI plans, opens u..."
5,'Godfather of artificial intelligence' says AI...
6,Microsoft is allegedly working with AMD to exp...
7,katze
